In [ ]:
%load_ext autoreload
%autoreload 2

<div class="main-title">
<h1>Geospatial data</h1>
<p>Loaders<p>
</div>

### Loaders

* used to load spatial data from different sources
* unify loading into a single interface
* prepare data for the embedding methods

[API Documentation](https://srai-lab.github.io/srai/latest/api/loaders/)<br>
[Examples](https://srai-lab.github.io/srai/latest/examples/loaders/)

### Types of loaders:

* GeoparquetLoader
* GTFSLoader
* OSMOnlineLoader
* OSMPbfLoader
* OSMWayLoader
* OSMTileLoader

Specify the city

In [ ]:
CITY = "Basel"
COUNTRY = "Switzerland"

area_name = f"{CITY}, {COUNTRY}"
area_name

Download the area

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

area = geocode_to_region_gdf(area_name)
area.explore(height=500)

## GeoparquetLoader

blah

## GTFSLoader

blah2

## BLAHLoader

blah3

## OSMPbfLoader


OSM(OpenStreetMap)[1] PBF(Protocolbuffer Binary Format)[2] loader is a loader capable of loading OSM features from a PBF file. It filters features based on OSM tags[3] in form of key:value pairs, that are used by OSM users to give meaning to geometries.

This loader uses pyosmium[3] library capable of parsing an *.osm.pbf file.

Additionally, it can download a pbf file extract for a given area using Protomaps API.

---

OSMPbfLoader can really quickly parse full OSM extract in the form of *.osm.pbf file.

It can download and parse a lot of features much faster than the OSMOnlineLoader, but it's much more useful when a lot of different features are required at once (like when using predefined filters).

When only a single or few features are needed, OSMOnlineLoader might be a better choice, since OSMPbfLoader will use a full extract of all features in a given region and will have to iterate over all of them.

Download the features

In [ ]:
from srai.loaders import OSMPbfLoader
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS

tags = {
    k: v
    for k, v in GEOFABRIK_LAYERS.items()
    if k in ["water", "waterways", "major_roads", "minor_roads"]
}
features = OSMPbfLoader().load(area, tags).clip(area)
features

In [ ]:
import matplotlib.pyplot as plt
from typing import Tuple
import geopandas as gpd


def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]


def plot_rectangle_with_text(
    ax: plt.Axes,
    coords: Tuple[float, float],
    title: str,
    subtitle: str = "",
):
    width = 1.0
    height = 0.085
    fontsize_title = 45
    fontsize_subtitle = 15

    rectangle = plt.Rectangle(
        coords,
        width,
        height,
        facecolor="#ecedea",
        alpha=0.8,
        transform=ax.transAxes,
        zorder=2,
    )

    ax.add_patch(rectangle)
    rx, ry = rectangle.get_xy()
    cx = rx + rectangle.get_width() / 2.0
    cy = ry + rectangle.get_height() / 2.0

    ax.text(
        cx,
        cy,
        title,
        fontsize=fontsize_title,
        transform=ax.transAxes,
        horizontalalignment="center",
        verticalalignment="center",
        color="#2b2b2b",
    )

    ax.text(
        cx,
        cy - 0.032,
        subtitle,
        fontsize=fontsize_subtitle,
        transform=ax.transAxes,
        horizontalalignment="center",
        verticalalignment="center",
        color="#2b2b2b",
    )


def plot_poster(gdf: gpd.GeoDataFrame) -> plt.axes:
    centroid = gdf.dissolve().centroid.item()
    lat = dd2dms(centroid.y)
    lng = dd2dms(centroid.x)

    fig = plt.figure(figsize=(8.27, 11.69))
    ax = fig.add_subplot()
    ax.set_position([0, 0, 1, 1])

    gdf.dropna(subset=["water", "waterways"], how="all").plot(ax=ax, color="#a8e1e6")

    gdf.dropna(subset=["major_roads", "minor_roads"], how="all").plot(
        ax=ax, color="#181818", markersize=0.1
    )

    plot_rectangle_with_text(
        ax,
        (0, 0.90),
        f"{abs(lat[0])}°{lat[1]}' {'N' if centroid.y >= 0 else 'S'}, {abs(lng[0])}°{lng[1]}' {'E' if centroid.x >= 0 else 'W'}",
    )
    plot_rectangle_with_text(ax, (0, 0.15), CITY, COUNTRY)

    xmin, ymin, xmax, ymax = gdf.total_bounds
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    ax.set_axis_off()
    ax.add_patch(
        plt.Rectangle(
            (0, 0), 1, 1, facecolor="#ecedea", transform=ax.transAxes, zorder=-1
        )
    )

    ax.margins(0, 0)

    return ax

Prepare the poster

In [ ]:
ax = plot_poster(features)

plt.savefig("poster.png", facecolor="#ecedea", dpi=300)
plt.close()

<img src="poster.png" style="height:800px;margin:auto" />

See also https://github.com/marceloprates/prettymaps for more poster inspiration!

# Thank You